# Una visión general de `astropy.coordinates`:

### Cruce de catálogos, planificación de observaciones y manejo de velocidades estelares

Los astrónomos utilizan una gran variedad de sistemas y formatos de coordenadas para representar las posiciones en el cielo de objetos celestes. Por ejemplo, quizás estés familiarizado con términos como “ascensión recta” y “declinación” o “latitud/longitud galáctica”, y tal vez hayas visto coordenadas angulares representadas como “0h39m15.9s”, “00:39:15.9” o 9.81625º. El módulo `astropy.coordinates` proporciona herramientas para representar coordenadas de objetos y transformarlas entre distintos sistemas.

En este tutorial, exploraremos cómo el paquete `astropy.coordinates` y otras funcionalidades relacionadas de Astropy pueden ayudarte en la planificación de observaciones u otras tareas centradas en catálogos de coordenadas extensos.

Puede ser útil tener abierta [la documentación de Astropy para el paquete de coordenadas](http://docs.astropy.org/en/stable/coordinates/index.html) mientras trabajas con este tutorial como referencia o para lecturas adicionales. En el texto a continuación, también verás algunos enlaces como ([docs](http://docs.astropy.org/en/stable/coordinates/index.html)). Estos te llevarán directamente a partes de la documentación relevantes para la celda desde la que provienen.

Algunos de los ejercicios a continuación son bastante complejos de resolver. Por esta razón, proporcionamos una posible solución para cada uno (¡aunque hay muchas respuestas posibles correctas!). También ten en cuenta que probablemente no haya suficiente tiempo para completar *todos* los ejercicios durante el tiempo disponible. Así que si te quedas atascado o no estás interesado en un ejercicio, te animamos a que lo omitas y regreses solo si más adelante tienes tiempo.

### Importaciones preliminares

Comenzamos importando algunos paquetes generales que necesitaremos más adelante:


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

Nota: Si la siguiente celda falla, asegúrese de haber instalado `astroquery` ejecutando:
```
pip install astroquery
```
en su terminal. Si es así, deberá ir a "Kernel -> Reiniciar" en este notebook después de instalarlo.

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.io import fits
from astropy.table import QTable
from astropy.utils.data import download_file

### Prefacio: representación del tiempo

¡Mantener el tiempo puede ser un asunto complicado, pero astropy puede facilitarlo! Existen muchas unidades diferentes que podrías necesitar usar, y muchas convenciones distintas para representar esos tiempos, todas implementadas en astropy.

Primero, importamos el objeto `Time` y representamos un momento utilizando una fecha Juliana:

In [ ]:
from astropy.time import Time

t = Time(2455000, format='jd')

## Alternativamente, puede inicializar el mismo momento con:
# t = Time('2009-06-17 12:00')

t

Puede ver que por defecto la escala de tiempo está en UTC, el formato proporcionado fue JD y se representa en ese mismo formato a la derecha.

Puede convertir fácilmente entre formatos de tiempo llamando al nombre del formato deseado como un atributo del objeto de tiempo, así:

In [ ]:
t.jd, t.iso, t.datetime

Supongamos que desea convertir a una escala diferente, por ejemplo de UTC a UT1, simplemente agregue `ut1` como atributo:

In [ ]:
t.ut1.iso

Puede inicializar rápidamente un arreglo vectorial de tiempos con 10 valores linealmente espaciados, comenzando en el tiempo actual, de la siguiente manera:

In [ ]:
times = Time.now() + np.linspace(0, 1, 10) * u.day
times

## Representación de posiciones en el cielo con `astropy.coordinates`

En Astropy, la forma más común de representar y trabajar con coordenadas celestes es mediante el objeto `SkyCoord`. Un objeto `SkyCoord` puede crearse directamente a partir de ángulos o arreglos de ángulos con sus unidades asociadas, como se muestra a continuación.

Para comenzar, supongamos que queremos crear un objeto `SkyCoord` para el centro del cúmulo abierto NGC 188, de modo que más adelante podamos consultar y recuperar estrellas que podrían ser miembros del cúmulo. Supongamos también, por ahora, que ya conocemos las coordenadas celestes del cúmulo: (12.11, 85.26) grados en el sistema de coordenadas ICRS. El ICRS —a veces llamado sistema “ecuatorial” o “J2000”— es actualmente el sistema de coordenadas astronómicas más común para la astronomía estelar o extragaláctica, y es el sistema predeterminado de coordenadas para `SkyCoord`. Por lo tanto, podríamos crear un objeto `SkyCoord` para el centro de NGC 188 así:

In [ ]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg)
ngc188_center

Se recomienda en general especificar explícitamente el sistema de referencia en el que se encuentran sus coordenadas. En este caso, dado que ICRS es el sistema predeterminado, esta sería una forma equivalente de crear nuestro objeto `SkyCoord`:

In [ ]:
ngc188_center = SkyCoord(12.11*u.deg, 85.26*u.deg, frame='icrs')
ngc188_center

Como veremos más adelante, existen muchos otros sistemas de coordenadas compatibles.

En las inicializaciones anteriores, proporcionamos objetos `astropy.units.Quantity` con unidades angulares para especificar los componentes angulares de nuestras coordenadas celestes. Sin embargo, `SkyCoord` también acepta coordenadas en formato de cadena de texto, ya sea como cadenas separadas para Ascensión Recta (RA) y Declinación (Dec), o como una sola cadena:

In [ ]:
SkyCoord('00h48m26.4s', '85d15m36s', frame='icrs')

Algunas representaciones en cadena de texto no definen explícitamente las unidades, por lo que deberá especificarlas de forma explícita si no están incluidas en la cadena de texto:

In [ ]:
SkyCoord('00:48:26.4 85:15:36', unit=(u.hour, u.deg), 
         frame='icrs')

Para más información y ejemplos sobre cómo inicializar objetos `SkyCoord`, [consulte esta documentación](http://docs.astropy.org/en/latest/coordinates/skycoord.html).

Para las inicializaciones anteriores, asumimos que ya teníamos los valores de las componentes de coordenadas disponibles. Si no conoce los valores de las coordenadas, y el objeto que le interesa se encuentra en [SESAME](http://cdsweb.u-strasbg.fr/cgi-bin/Sesame), también puede buscar y cargar automáticamente las coordenadas a partir del nombre del objeto utilizando el método de clase `SkyCoord.from_name()`<sup>1</sup> ([documentación](http://docs.astropy.org/en/latest/coordinates/index.html#convenience-methods)). Tenga en cuenta, sin embargo, que esto requiere una conexión a internet. Puede omitir esta celda con seguridad si no está conectado, ya que el objeto ya fue definido anteriormente.

<sub><sup>1</sup>Si no sabe qué es un método de clase, puede pensarlo como un constructor alternativo para un objeto `SkyCoord`: al llamar `SkyCoord.from_name()` con un nombre, se genera un nuevo objeto `SkyCoord`. Para una explicación más detallada sobre qué son los métodos de clase y cuándo son útiles, consulte [esta página](https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods).</sub>

In [ ]:
# NOTA: omita esta celda si no tiene conexión a internet

ngc188_center = SkyCoord.from_name('NGC 188')
ngc188_center

El objeto `SkyCoord` que creamos tiene varias formas de acceder a la información que contiene. Todos los objetos `SkyCoord` tienen atributos que le permiten recuperar los datos de las componentes de coordenadas, pero los nombres de las componentes cambiarán dependiendo del sistema de coordenadas del `SkyCoord` que tenga. 

En nuestro caso, tenemos un `SkyCoord` con coordenadas en el sistema ICRS: por lo tanto, los nombres de las componentes son abreviaciones en minúsculas de Ascensión Recta, `.ra`, y Declinación, `.dec`:

In [ ]:
ngc188_center.ra, ngc188_center.dec

Los atributos ``ra`` y ``dec`` devuelven objetos especializados de tipo [``Quantity``](http://docs.astropy.org/en/stable/units/index.html) (en realidad, una subclase llamada [``Angle``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html), que a su vez es subclasificada por [``Latitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Latitude.html) y [``Longitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html)).

Estos objetos almacenan ángulos y proporcionan representaciones elegantes de esos ángulos, además de atributos útiles para convertir rápidamente a unidades angulares comunes. Por ejemplo, en un notebook de Jupyter, estos objetos saben cómo representarse a sí mismos utilizando LaTeX:

In [ ]:
ngc188_center.ra

In [ ]:
ngc188_center.dec

También podemos obtener los valores de las componentes en diferentes unidades:

In [ ]:
ngc188_center.ra.hour, ngc188_center.ra.radian, ngc188_center.ra.degree

Y podemos formatear los valores en cadenas de texto con unidades específicas ([documentación](http://docs.astropy.org/en/latest/coordinates/formatting.html)), por ejemplo:

In [ ]:
ngc188_center.ra.to_string(unit=u.hourangle, sep=':')

Ahora que tenemos un objeto `SkyCoord` para el centro de NGC 188, podemos seleccionar fuentes del catálogo de la segunda entrega de datos de *Gaia* (*Gaia* Data Release 2) alrededor de esta posición, para intentar identificar estrellas que podrían ser miembros del cúmulo.

Para ello, utilizaremos el módulo `astroquery.gaia` para consultar el archivo de datos de *Gaia* ([documentación](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)).

Esto requiere una conexión a internet. Sin embargo, si la consulta falla, el archivo del catálogo está incluido en el repositorio, por lo que puedes cargarlo localmente (es decir, omite la siguiente celda si no tienes conexión a internet):


In [ ]:
from astroquery.gaia import Gaia

In [ ]:
# NOTA: omite esta celda si no tiene conexión a internet

job = Gaia.cone_search_async(ngc188_center, radius=0.5*u.deg)
table = job.get_results()

# conservar solo las estrellas más brillantes que magnitud G = 19
table = table[table['phot_g_mean_mag'] < 19*u.mag]

La celda anterior puede que no funcione si no tiene conexión a internet, por lo que hemos incluido la tabla de resultados junto con el notebook:

In [ ]:
table = QTable.read('gaia_results.fits')

In [ ]:
len(table)

Esta tabla ahora contiene aproximadamente 5000 estrellas del catálogo *Gaia* DR2 alrededor de la posición de coordenadas del centro de NGC 188. 

Ahora construyamos un objeto `SkyCoord` a partir de la tabla de resultados. En el archivo de datos de *Gaia*, las coordenadas ICRS de una fuente se proporcionan en las columnas con los nombres `"ra"` y `"dec"`:

In [ ]:
table['ra']

In [ ]:
table['dec']

Tenga en cuenta que, dado que el archivo de *Gaia* proporciona tablas de datos con unidades asociadas, y leímos esta tabla utilizando el objeto `QTable` ([documentación](http://docs.astropy.org/en/latest/table/mixin_columns.html#quantity-and-qtable)), las columnas mencionadas están representadas como objetos `Quantity` con unidades en grados.

También es importante notar que estas columnas contienen muchas coordenadas (¡más de 5000!). Podemos pasar estos valores directamente a `SkyCoord` para obtener un solo objeto `SkyCoord` que represente todas estas coordenadas:

In [ ]:
gaia_coords = SkyCoord(table['ra'], table['dec'])
gaia_coords

### Ejercicios

Crea un objeto `SkyCoord` del cúmulo abierto de las Pléyades (puedes buscar las coordenadas y pasarlas directamente, o usar el método conveniente que aprendimos anteriormente):

Usando únicamente una sola llamada a un método/función en el objeto `SkyCoord` que representa el centro de NGC 188, imprime una cadena con las coordenadas RA/Dec en el formato 'HH:MM:SS.S DD:MM:SS.S'. Verifica tu respuesta en [SIMBAD](http://simbad.u-strasbg.fr/simbad/), donde se muestran las coordenadas en formato sexagesimal para el objeto.

<sub>(Pista: `SkyCoord.to_string()` podría ser útil.)</sub>

Usando una sola llamada a un método/función en el objeto `SkyCoord` que contiene los resultados de nuestra consulta a *Gaia*, calcula la separación angular entre cada estrella resultante y las coordenadas del centro del cúmulo.

<sub>(Pista: `SkyCoord.separation()` podría ser útil.)</sub>

---

## Más que solo posiciones en el cielo: Incluir información de distancia en `SkyCoord`

Hasta ahora, solo hemos usado `SkyCoord` para representar posiciones en el cielo (es decir, únicamente `ra` y `dec`). Sin embargo, a veces es útil incluir información de distancia junto con las coordenadas celestes, especificando así completamente la posición 3D de una fuente dada. Para incluir la información de distancia, `SkyCoord` acepta un argumento con palabra clave llamado `distance`. Por ejemplo, si conociéramos la distancia al cúmulo NGC 188, también podríamos pasarla (como un objeto `Quantity`) usando este argumento:

In [ ]:
ngc188_center_3d = SkyCoord(12.11*u.deg, 85.26*u.deg,
                            distance=1.96*u.kpc)

Con los resultados de nuestra consulta al catálogo de *Gaia*, también se nos proporcionan mediciones de paralaje para cada estrella del catálogo. La forma predeterminada de proporcionar una distancia, como se mostró anteriormente, es pasando un objeto `Quantity` con una unidad de longitud. Sin embargo, `astropy.coordinates` también ofrece un objeto especializado llamado `Distance`, para manejar transformaciones comunes entre diferentes representaciones de distancia ([documentación](http://docs.astropy.org/en/latest/coordinates/index.html#distance)). Entre otras cosas, esta clase permite proporcionar un valor de paralaje (¡pero ten en cuenta que la transformación a distancia aquí invierte la paralaje, lo cual no siempre es una buena idea!):

In [ ]:
Distance(parallax=1*u.mas)

El catálogo de estrellas que consultamos de *Gaia* contiene información de paralaje, por lo que también podemos crear un objeto `SkyCoord` para representar las posiciones 3D de todas las estrellas de *Gaia*. Aquí, hacemos una subselección de la tabla para devolver únicamente las estrellas con valores positivos de paralaje:

In [ ]:
table_3d = table[table['parallax'] > 0]

gaia_dist = Distance(parallax=table_3d['parallax'])
gaia_coords_3d = SkyCoord(table_3d['ra'], table_3d['dec'],
                          distance=gaia_dist)
gaia_coords_3d

Ahora que tenemos información de posición 3D tanto para el centro del cúmulo como para las estrellas que consultamos de *Gaia*, podemos calcular la separación 3D (distancia) entre todas las fuentes de *Gaia* y el centro del cúmulo:

In [ ]:
sep3d = gaia_coords_3d.separation_3d(ngc188_center_3d)
sep3d

### Ejercicios

Usando los valores de separación 3D, define una máscara booleana para seleccionar los miembros candidatos del cúmulo. Selecciona todas las estrellas que estén dentro de 50 pc del centro del cúmulo. ¿Cuántos miembros candidatos de NGC 188 tenemos, basándonos en sus posiciones 3D?

## Material Extra

Recapitulemos lo que hemos hecho hasta ahora: definimos un objeto `SkyCoord` para representar el centro de un cúmulo abierto (NGC 188), consultamos el catálogo *Gaia* DR2 para seleccionar estrellas que están cerca (en el cielo) del centro del cúmulo, y usamos los valores de paralaje de *Gaia* para seleccionar estrellas que están cerca de NGC 188 en posición 3D.

Este tutorial cubre solo una introducción muy breve al uso de `astropy.coordinates`, pero el módulo tiene aún más funcionalidades que no pudimos abordar en este taller. Para consultar la documentación sobre otras características de `astropy.coordinates`, visita [la sección de astropy.coordinates en la documentación de Astropy](http://astropy.readthedocs.org/en/stable/coordinates/index.html).